In [29]:
using Laplacians

In [30]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgVBLASMatNorm (generic function with 1 method)

In [31]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [32]:
a = readFromFile("../graphs/se_3/graph.mtx");

In [33]:
tree = readFromFile("../graphs/se_3/tree.mtx");

In [34]:
x = readFromFile("../graphs/se_3/x.vec");

In [35]:
# b = readFromFile("../graphs/se_1/b.vec");
b = lap(a) * x;

In [36]:
norm(lap(a) * x - b) / norm(b)

0.0

In [37]:
la = lap(a);

In [38]:
ts = treeSolver(tree)

F = function(b)
    return ts(b - mean(b))
end

(anonymous function)

In [20]:
@time myx64 = pcg(la, b, F, tol=1e-1, maxits=100000, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)

LoadError: LoadError: InterruptException:
while loading In[20], in expression starting on line 155

In [21]:
dbg = []

for hp in [64,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    tsHP = treeSolver(treeHP)

    FHP = function(b)
        return tsHP(b - mean(b))
    end
    
    push!(dbg, "======= $(hp) bit precision")
    @time myxHP = pcgVMatNorm(laHP, bHP, FHP, xHP, tol=BigFloat(1e-2), maxits=100000, verbose=true);
#     @time myxHP = pcgV(laHP, bHP, FHP, tol=BigFloat(1e-3), maxits=100000, verbose=true);
    println("Error is: ", Float64(norm(laHP * myxHP - bHP) / norm(bHP)))
    push!(dbg, "")
    push!(dbg, "")
    
end

LoadError: LoadError: InterruptException:
while loading In[21], in expression starting on line 3

In [22]:
writeToFile("../graphs/se_3/_log_julia.txt", dbg)

LoadError: LoadError: BoundsError: attempt to access 0-element Array{Any,1}
  at index [1]
while loading In[22], in expression starting on line 1